In [1]:
import os
import h5py
import numpy as np
import tensorflow as tf
from tqdm import tqdm


In [2]:
import glob
import os


In [3]:
CKPT_DIR = "./SavedModels_RSCA_GAN_full_2"
BATCH_SIZE = 8

val_folder = r"D:\fastmri_singlecoil_FSSCAN\val_norm"
TEST_FILES = sorted(
    glob.glob(os.path.join(val_folder, "*.h5"))
)


In [41]:
pd_files = []
pdfs_files = []

for f in TEST_FILES:
    if "PDFS" in f:
        pdfs_files.append(f)
    else:
        pd_files.append(f)

print(f"PD volumes: {len(pd_files)}")
print(f"PDFS volumes: {len(pdfs_files)}")


PD volumes: 100
PDFS volumes: 99


In [4]:
%run model.ipynb

In [5]:
model = RSCAGAN(in_channels=2, base_channels=32)
generator = model.generator
discriminator = model.discriminator  # required for ckpt restore

ckpt = tf.train.Checkpoint(
    generator=generator,
    discriminator=discriminator
)

ckpt_manager = tf.train.CheckpointManager(
    ckpt,
    directory=CKPT_DIR,
    max_to_keep=5
)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint).expect_partial()
    print(f"✅ Loaded checkpoint: {ckpt_manager.latest_checkpoint}")
else:
    raise RuntimeError("❌ No checkpoint found")


✅ Loaded checkpoint: ./SavedModels_RSCA_GAN_full_2\ckpt-14


In [6]:
def complex_to_mag_np(x):
    return np.sqrt(x[..., 0]**2 + x[..., 1]**2 + 1e-8)


In [7]:
def nmse(gt, pred):
    return np.linalg.norm(gt - pred) ** 2 / (np.linalg.norm(gt) ** 2 + 1e-10)


In [8]:
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
def compute_ssim(gt, pred, max_val):
    return structural_similarity(
        gt, pred,
        data_range=max_val,
        win_size=9,
        gaussian_weights=False,
        use_sample_covariance=False,
        K1=0.01,
        K2=0.03
    )


In [9]:
@tf.function
def inference_step(SZF):
    return generator(SZF, training=False)


In [15]:
import os
import numpy as np
import h5py
import pandas as pd
import tensorflow as tf
from tqdm import tqdm

def run_inference_and_save(test_files, out_dir="./Metrics"):
    os.makedirs(out_dir, exist_ok=True)

    # -------------------------
    # Containers for Excel
    # -------------------------
    volume_rows = []   # per-volume metrics
    slice_rows  = []   # slice-wise SSIM

    for file_path in tqdm(test_files, desc="Inference (per-volume)"):
        volume_id = os.path.basename(file_path)

        with h5py.File(file_path, "r") as f:
            SZF  = f["image_under"][:]            # (N,H,W,2)
            SGT  = f["image_full"][:]             # (N,H,W,2)
            MAXV = f["max_val_full_image"][0]     # scalar

        slice_psnr = []
        slice_ssim = []
        slice_nmse = []

        slice_counter = 0

        for i in range(0, SZF.shape[0], BATCH_SIZE):
            SZF_batch = SZF[i:i+BATCH_SIZE]
            SGT_batch = SGT[i:i+BATCH_SIZE]

            SRE_batch = inference_step(
                tf.convert_to_tensor(SZF_batch, tf.float32)
            ).numpy()

            # -------------------------
            # Magnitude + denormalize
            # -------------------------
            gt_mag  = complex_to_mag_np(SGT_batch) * MAXV
            rec_mag = complex_to_mag_np(SRE_batch) * MAXV

            for b in range(gt_mag.shape[0]):
                gt  = gt_mag[b]
                rec = rec_mag[b]

                # PSNR
                psnr_val = tf.image.psnr(
                    gt[..., None], rec[..., None], max_val=MAXV
                ).numpy()
                slice_psnr.append(psnr_val)

                # SSIM
                ssim_val = compute_ssim(gt, rec, max_val=MAXV)
                slice_ssim.append(ssim_val)

                # NMSE
                nmse_val = nmse(gt, rec)
                slice_nmse.append(nmse_val)

                # -------------------------
                # Save slice-wise SSIM row
                # -------------------------
                slice_rows.append({
                    "volume_id": volume_id,
                    "slice_id": slice_counter,
                    "SSIM": float(ssim_val)
                })

                slice_counter += 1

        # -------------------------
        # Per-volume aggregation
        # -------------------------
        volume_rows.append({
            "volume_id": volume_id,
            "PSNR": float(np.mean(slice_psnr)),
            "NMSE": float(np.mean(slice_nmse)),
            "SSIM_mean": float(np.mean(slice_ssim)),
        })

    # =========================
    # Save Excel files
    # =========================
    df_volume = pd.DataFrame(volume_rows)
    df_slice  = pd.DataFrame(slice_rows)

    volume_excel_path = os.path.join(out_dir, "RSCA_per_volume_metrics.xlsx")
    slice_excel_path  = os.path.join(out_dir, "RSCA_slice_wise_ssim.xlsx")

    df_volume.to_excel(volume_excel_path, index=False)
    df_slice.to_excel(slice_excel_path, index=False)

    print(f"Saved per-volume metrics to: {volume_excel_path}")
    print(f"Saved slice-wise SSIM to:   {slice_excel_path}")

    return df_volume, df_slice


In [16]:
_,_  = run_inference_and_save(TEST_FILES)

print("\n📊 RSCA-GAN INFERENCE RESULTS (Per-Volume)")
print(f" PSNR : {results['PSNR_mean']:.2f} ± {results['PSNR_std']:.2f}")
print(f" SSIM : {results['SSIM_mean']:.4f} ± {results['SSIM_std']:.4f}")
print(f" NMSE : {results['NMSE_mean']:.6f} ± {results['NMSE_std']:.6f}")


Inference (per-volume): 100%|████████████████████████████████████████████████████████| 199/199 [05:36<00:00,  1.69s/it]


Saved per-volume metrics to: ./Metrics\RSCA_per_volume_metrics.xlsx
Saved slice-wise SSIM to:   ./Metrics\RSCA_slice_wise_ssim.xlsx

📊 RSCA-GAN INFERENCE RESULTS (Per-Volume)


NameError: name 'results' is not defined

In [44]:
def run_inference(test_files):
    vol_psnr = []
    vol_ssim = []
    vol_nmse = []

    for file_path in tqdm(test_files, desc="Inference (per-volume)"):
        with h5py.File(file_path, "r") as f:
            SZF  = f["image_under"][:]            # (N,H,W,2)
            SGT  = f["image_full"][:]             # (N,H,W,2)
            MAXV = f["max_val_full_image"][0]     # scalar

        slice_psnr = []
        slice_ssim = []
        slice_nmse = []

        for i in range(0, SZF.shape[0], BATCH_SIZE):
            SZF_batch = SZF[i:i+BATCH_SIZE]
            SGT_batch = SGT[i:i+BATCH_SIZE]

            SRE_batch = inference_step(
                tf.convert_to_tensor(SZF_batch, tf.float32)
            ).numpy()

            # -------------------------
            # Magnitude + denormalize
            # -------------------------
            gt_mag  = complex_to_mag_np(SGT_batch) * MAXV
            rec_mag = complex_to_mag_np(SRE_batch) * MAXV

            for b in range(gt_mag.shape[0]):
                gt  = gt_mag[b]
                rec = rec_mag[b]

                # PSNR
                slice_psnr.append(
                    tf.image.psnr(
                        gt[..., None], rec[..., None], max_val=MAXV
                    ).numpy()
                )

                # SSIM (your implementation)
                slice_ssim.append(
                    compute_ssim(gt, rec, max_val=MAXV)
                )

                # NMSE (your implementation)
                slice_nmse.append(
                    nmse(gt, rec)
                )

        # -------------------------
        # Per-volume aggregation
        # -------------------------
        vol_psnr.append(np.mean(slice_psnr))
        vol_ssim.append(np.mean(slice_ssim))
        vol_nmse.append(np.mean(slice_nmse))

    return {
        "PSNR_mean": float(np.mean(vol_psnr)),
        "PSNR_std":  float(np.std(vol_psnr)),
        "SSIM_mean": float(np.mean(vol_ssim)),
        "SSIM_std":  float(np.std(vol_ssim)),
        "NMSE_mean": float(np.mean(vol_nmse)),
        "NMSE_std":  float(np.std(vol_nmse)),
    }


In [45]:
results = run_inference(pdfs_files)

print("\n📊 RSCA-GAN INFERENCE RESULTS (Per-Volume)")
print(f" PSNR : {results['PSNR_mean']:.2f} ± {results['PSNR_std']:.2f}")
print(f" SSIM : {results['SSIM_mean']:.4f} ± {results['SSIM_std']:.4f}")
print(f" NMSE : {results['NMSE_mean']:.6f} ± {results['NMSE_std']:.6f}")


Inference (per-volume): 100%|██████████████████████████████████████████████████████████| 99/99 [02:40<00:00,  1.62s/it]


📊 RSCA-GAN INFERENCE RESULTS (Per-Volume)
 PSNR : 33.09 ± 2.75
 SSIM : 0.7771 ± 0.0795
 NMSE : 0.040090 ± 0.016491


In [39]:
import os
import time
import glob
import psutil
import numpy as np
import h5py
from tqdm import tqdm
import tensorflow as tf
from keras_flops import get_flops

# ============================================================
# CONFIGURATION
# ============================================================
VAL_FOLDER = r"D:\fastmri_singlecoil_FSSCAN\val_norm"
CKPT_DIR   = r"./SavedModels_RSCA_GAN_full_2"   # 🔴 CHANGE THIS
IMG_H, IMG_W = 320, 320                 # FastMRI resolution
WARMUP_SLICES = 10
NUM_TIMING_SLICES = 100                 # fixed slice budget

# ============================================================
# FILE LIST
# ============================================================
file_paths = sorted(glob.glob(os.path.join(VAL_FOLDER, "*.h5")))

# ============================================================
# LOAD RSCA-GAN MODEL
# ============================================================
model = RSCAGAN(in_channels=2, base_channels=32)
generator = model.generator
discriminator = model.discriminator  # required only for checkpoint restore

ckpt = tf.train.Checkpoint(
    generator=generator,
    discriminator=discriminator
)

ckpt_manager = tf.train.CheckpointManager(
    ckpt,
    directory=CKPT_DIR,
    max_to_keep=5
)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint).expect_partial()
    print(f"✅ Loaded checkpoint: {ckpt_manager.latest_checkpoint}")
else:
    raise RuntimeError("❌ No checkpoint found")

# ============================================================
# BUILD GENERATOR (REQUIRED FOR SUBCLASSED MODELS)
# ============================================================
dummy_input = tf.zeros((1, IMG_H, IMG_W, 2), dtype=tf.float32)
_ = generator(dummy_input, training=False)

# ============================================================
# PARAMETER COUNT (GENERATOR ONLY)
# ============================================================
num_params = generator.count_params()

# ============================================================
# FLOPs (GENERATOR ONLY, PER SLICE)
# Workaround for subclassed models
# ============================================================
def build_functional_generator(gen, input_shape):
    inp = tf.keras.Input(shape=input_shape)
    out = gen(inp, training=False)
    return tf.keras.Model(inputs=inp, outputs=out)

gen_for_flops = build_functional_generator(
    generator,
    input_shape=(IMG_H, IMG_W, 2)
)

flops = get_flops(gen_for_flops, batch_size=1)

# ============================================================
# INFERENCE FUNCTION (DEPLOYMENT PATH)
# ============================================================
@tf.function
def inference_step(x):
    return generator(x, training=False)

# ============================================================
# MEMORY HELPERS
# ============================================================
process = psutil.Process(os.getpid())

def cpu_memory_mb():
    return process.memory_info().rss / (1024 ** 2)

def gpu_memory_mb():
    info = tf.config.experimental.get_memory_info("GPU:0")
    return info["peak"] / (1024 ** 2)

# ============================================================
# LATENCY / THROUGHPUT MEASUREMENT
# ============================================================
def measure_latency(device):

    latencies = []

    with tf.device(device):

        # -----------------------------
        # WARM-UP
        # -----------------------------
        for file in file_paths[:1]:
            with h5py.File(file, "r") as f:
                image_under = f["image_under"][:]

            for s in range(min(WARMUP_SLICES, image_under.shape[0])):
                x = tf.convert_to_tensor(image_under[s:s+1], tf.float32)
                _ = inference_step(x)

        # -----------------------------
        # TIMED INFERENCE
        # -----------------------------
        count = 0
        for file in tqdm(file_paths, desc=f"Timing on {device}", ncols=120):

            with h5py.File(file, "r") as f:
                image_under = f["image_under"][:]

            for s in range(image_under.shape[0]):

                if count >= NUM_TIMING_SLICES:
                    break

                x = tf.convert_to_tensor(image_under[s:s+1], tf.float32)
                assert x.shape[0] == 1

                start = time.perf_counter()
                _ = inference_step(x)

                # 🔑 GPU synchronization
                if "GPU" in device:
                    tf.config.experimental.get_memory_info("GPU:0")

                end = time.perf_counter()

                latencies.append(end - start)
                count += 1

            if count >= NUM_TIMING_SLICES:
                break

    latencies = np.array(latencies)
    mean_s = latencies.mean()

    return {
        "mean_s": mean_s,
        "std_s": latencies.std(),
        "slices_per_sec": 1.0 / mean_s
    }

# ============================================================
# CPU BENCHMARK
# ============================================================
cpu_mem_before = cpu_memory_mb()
cpu_latency = measure_latency("/CPU:0")
cpu_mem_after = cpu_memory_mb()
cpu_mem_peak = cpu_mem_after - cpu_mem_before

# ============================================================
# GPU BENCHMARK (IF AVAILABLE)
# ============================================================
gpu_latency = None
gpu_mem_peak = None
gpu_name = None

gpus = tf.config.list_physical_devices("GPU")
if gpus:
    tf.config.experimental.reset_memory_stats("GPU:0")
    gpu_name = tf.config.experimental.get_device_details(gpus[0])["device_name"]
    gpu_latency = measure_latency("/GPU:0")
    gpu_mem_peak = gpu_memory_mb()

# ============================================================
# FINAL REPORT
# ============================================================
print("\n" + "=" * 70)
print("RSCA-GAN GENERATOR — EFFICIENCY REPORT (BATCH SIZE = 1)")
print("=" * 70)

print(f"Parameters (Generator): {num_params / 1e6:.2f} M")
print(f"FLOPs (per slice):      {flops / 1e9:.2f} GFLOPs")

print("\n--- CPU Inference ---")
print(f"Latency:     {cpu_latency['mean_s']:.2f} s / slice")
print(f"Throughput:  {cpu_latency['slices_per_sec']:.3f} slices/sec")
print(f"Memory:      {cpu_mem_peak:.2f} MB")

if gpu_latency:
    print("\n--- GPU Inference ---")
    print(f"GPU:         {gpu_name}")
    print(f"Latency:     {gpu_latency['mean_s']:.2f} s / slice")
    print(f"Throughput:  {gpu_latency['slices_per_sec']:.3f} slices/sec")
    print(f"Peak VRAM:   {gpu_mem_peak:.2f} MB")
else:
    print("\nGPU not available.")

print("=" * 70)


✅ Loaded checkpoint: ./SavedModels_RSCA_GAN_full_2\ckpt-14
Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


Timing on /GPU:0:   1%|▋                                                                | 2/199 [00:04<06:41,  2.04s/it]


RSCA-GAN GENERATOR — EFFICIENCY REPORT (BATCH SIZE = 1)
Parameters (Generator): 30.00 M
FLOPs (per slice):      147.86 GFLOPs

--- CPU Inference ---
Latency:     0.94 s / slice
Throughput:  1.069 slices/sec
Memory:      246.88 MB

--- GPU Inference ---
GPU:         NVIDIA RTX A5000
Latency:     0.04 s / slice
Throughput:  27.816 slices/sec
Peak VRAM:   235.01 MB
